### 1. 라이브러리 불러오기

In [ ]:
import numpy as np
import torch
from torch import nn, optim
import pandas as pd

### 2. 데이터 준비하기

In [ ]:
train = pd.read_csv('./abalone_train.csv', header=None, skiprows=1)
test = pd.read_csv('./abalone_test.csv', header=None, skiprows=1)

input_cnt, output_cnt= 10, 1
train_df = np.zeros([len(train),input_cnt + output_cnt])
test_df = np.zeros([len(test),input_cnt + output_cnt])

### 3. 원핫인코딩

In [ ]:
# train data 원핫
for index, row in train.iterrows():
  if row[0] == 'I':
    train_df[index, 0] = 1
  if row[0] == 'M':
    train_df[index, 1] = 1
  if row[0] == 'F':
    train_df[index, 2] = 1

  train_df[:, 3:] = train.loc[:, 1:]

# test data 원핫
for index, row in test.iterrows():
  if row[0] == 'I':
    test_df[index, 0] = 1
  if row[0] == 'M':
    test_df[index, 1] = 1
  if row[0] == 'F':
    test_df[index, 2] = 1

  test_df[:, 3:] = test.loc[:, 1:]

### 4. 데이터형 변환

In [ ]:
# float 형으로 변환
train_df.astype('float')
test_df.astype('float')

array([[ 0.    ,  0.    ,  1.    , ...,  0.1905,  0.289 ,  9.    ],
       [ 0.    ,  1.    ,  0.    , ...,  0.196 ,  0.268 ,  8.    ],
       [ 0.    ,  1.    ,  0.    , ...,  0.2635,  0.335 , 12.    ],
       ...,
       [ 0.    ,  1.    ,  0.    , ...,  0.3765,  0.495 , 12.    ],
       [ 0.    ,  0.    ,  1.    , ...,  0.242 ,  0.22  ,  9.    ],
       [ 0.    ,  1.    ,  0.    , ...,  0.24  ,  0.255 , 10.    ]])

### 5. 데이터 자르기 

In [ ]:
# y : 정답 
# train_df split
train_df_len = train_df.shape[0]
train_X = torch.tensor(train_df[:, 0:-1], dtype=torch.float)
train_y = torch.tensor(train_df[:, [-1]], dtype=torch.float)

# test_df split
test_df_len = test_df.shape[0]
test_X = torch.tensor(test_df[:, 0:-1], dtype=torch.float)
test_y = torch.tensor(test_df[:, [-1]], dtype=torch.float)

### 6. 다층 퍼셉트론 구축

In [ ]:
model = nn.Sequential(
    nn.Linear(input_cnt, 15),
    nn.ReLU(),
    nn.Linear(15, output_cnt)
    )

### 7. 최적화

In [ ]:
optimizer = optim.RMSprop(model.parameters(), lr=0.01)

### 8. 손실함수

In [ ]:
# 손실함수 정의
loss_fn = nn.MSELoss()

# 손실함수 로그
losses = []

### 9. 학습

In [ ]:
for epoch in range(20):
  for idx in range(train_df_len):
    # 데이터 불러오기
    X = train_X[idx]
    y = train_y[idx]

    # 이전 학습의 계산된 경사 초기화
    optimizer.zero_grad()

    # 학습을 통한 예측값 계산
    y_pred =model(X)

    # 손실 함수 MSE(회귀 문제에서 주로 사용) 계산
    loss = loss_fn(y_pred, y)

    # 손실 함수로 구한 값으로 미분 계산
    loss.backward()

    # 경사 갱신(미분 갱신)
    optimizer.step()

    # loss 기록(수렴 확인용)
    losses.append(loss.item())
  print(loss)


tensor(0.3735, grad_fn=<MseLossBackward0>)
tensor(0.3183, grad_fn=<MseLossBackward0>)
tensor(0.3040, grad_fn=<MseLossBackward0>)
tensor(0.2437, grad_fn=<MseLossBackward0>)
tensor(0.2807, grad_fn=<MseLossBackward0>)
tensor(0.2339, grad_fn=<MseLossBackward0>)
tensor(0.2456, grad_fn=<MseLossBackward0>)
tensor(0.2750, grad_fn=<MseLossBackward0>)
tensor(0.2597, grad_fn=<MseLossBackward0>)
tensor(0.3418, grad_fn=<MseLossBackward0>)
tensor(0.2385, grad_fn=<MseLossBackward0>)
tensor(0.4525, grad_fn=<MseLossBackward0>)
tensor(0.4464, grad_fn=<MseLossBackward0>)
tensor(0.4186, grad_fn=<MseLossBackward0>)
tensor(0.5398, grad_fn=<MseLossBackward0>)
tensor(0.4921, grad_fn=<MseLossBackward0>)
tensor(0.4038, grad_fn=<MseLossBackward0>)
tensor(0.3525, grad_fn=<MseLossBackward0>)
tensor(0.3538, grad_fn=<MseLossBackward0>)
tensor(0.3565, grad_fn=<MseLossBackward0>)


### 10. 테스트 데이터를 통한 검증


In [ ]:
# test evaluation
test_pred = model(test_X)
diff = np.mean(np.abs(test_pred.detach().numpy() - test_y.numpy()))

# 출력
print(diff)

1.4103831
